# Creating an LLM agent

In this notebook, we'll create a simple agent that is able to call three tools:

- `execute_shell_command`,
- `read_file`,
- `file_write`.

To start with, let's import the required libraries and load API keys. To work with this notebook, you need to upload to Colab the `openai_api_key` and `nebius_api_key` files containing your OpenAI and Nebius AI Studio API keys.

In [ ]:
!pip install --force-reinstall -v -q "openai==1.55.3"

In [ ]:
import os

with open("openai_api_key", "r") as file:
    openai_api_key = file.read().strip()

os.environ["OPENAI_API_KEY"] = openai_api_key

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

Now, here is our agent class:

In [ ]:
import openai
import json
import subprocess
import os
from typing import List, Dict, Any
import shlex
from openai import OpenAI

class ShellAssistant:
    def __init__(self, client, model):
        """Initialize the assistant with your OpenAI API key."""
        self.model = model

        self.client = client

        # Define available tools
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "execute_shell_command",
                    "description": "Execute a shell command and return its output",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {
                                "type": "string",
                                "description": "The shell command to execute"
                            }
                        },
                        "required": ["command"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "read_file",
                    "description": "Read contents of a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {
                                "type": "string",
                                "description": "Path to the file to read"
                            }
                        },
                        "required": ["file_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "write_file",
                    "description": "Write content to a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {
                                "type": "string",
                                "description": "Path to the file to write"
                            },
                            "content": {
                                "type": "string",
                                "description": "Content to write to the file"
                            },
                            "mode": {
                                "type": "string",
                                "description": "Write mode: 'w' for overwrite, 'a' for append",
                                "enum": ["w", "a"],
                                "default": "w"
                            }
                        },
                        "required": ["file_path", "content"]
                    }
                }
            }
        ]

    def write_file(self, file_path: str, content: str, mode: str = 'w') -> Dict[str, Any]:
        """Write content to a file."""
        try:
            # Validate mode
            if mode not in ['w', 'a']:
                raise ValueError("Mode must be 'w' or 'a'")

            # Get absolute path and check if it's in a safe directory
            abs_path = os.path.abspath(file_path)

            # Add additional security checks here if needed
            # For example, restrict to specific directories

            with open(abs_path, mode) as f:
                f.write(content)

            return {
                "success": True,
                "message": f"Successfully wrote to {file_path}",
                "bytes_written": len(content)
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    def execute_shell_command(self, command: str) -> Dict[str, Any]:
        """Execute a shell command and return its output."""
        try:
            # Use shlex.split for proper command parsing
            args = shlex.split(command)

            # Execute command and capture output
            result = subprocess.run(
                args,
                capture_output=True,
                text=True,
                shell=False  # More secure than shell=True
            )

            return {
                "success": True,
                "stdout": result.stdout,
                "stderr": result.stderr,
                "return_code": result.returncode
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    def read_file(self, file_path: str) -> Dict[str, Any]:
        """Read a file and return its contents."""
        try:
            with open(file_path, 'r') as f:
                content = f.read()
            return {
                "success": True,
                "content": content
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    def process_tool_call(self, tool_call: Dict) -> Dict[str, Any]:
        """Process a tool call from the API response."""
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "execute_shell_command":
            return self.execute_shell_command(function_args["command"])
        elif function_name == "read_file":
            return self.read_file(function_args["file_path"])
        elif function_name == "write_file":
            mode = function_args.get("mode", "w")
            return self.write_file(
                function_args["file_path"],
                function_args["content"],
                mode
            )
        else:
            return {"success": False, "error": f"Unknown function: {function_name}"}

    def chat(self, user_message: str, verbose=False) -> str:
        """Main chat function that processes user input and returns assistant response."""
        completions = []
        messages = [{"role": "user", "content": user_message}]

        try:
            # Get initial response from OpenAI
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )

            # completions.append(completion)
            message = completion.choices[0].message

            # Process tool calls if any
            while message.tool_calls:
                messages.append(message)

                # Process each tool call
                for tool_call in message.tool_calls:
                    result = self.process_tool_call(tool_call)

                    # Add tool result to messages
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result)
                    })

                # Get next response from OpenAI
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    tools=self.tools,
                    tool_choice="auto"
                )
                # completions.append(completion)
                message = completion.choices[0].message

            if verbose:
                return message.content, messages#, completions
            else:
                return message.content

        except Exception as e:
            return f"Error: {str(e)}"



The following code creates the agent:

In [ ]:
from openai import OpenAI

# If you want to use OpenAI's GPT models:
assistant = ShellAssistant(
    client=OpenAI(),
    model="gpt-4o-mini"
    )

# If you want to use Nebius AI Studio:
'''client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

assistant = ShellAssistant(
    client=client,
    model="meta-llama/Meta-Llama-3.1-70B-Instruct")'''

'client = OpenAI(\n    base_url="https://api.studio.nebius.ai/v1/",\n    api_key=os.environ.get("NEBIUS_API_KEY"),\n)\n\nassistant = ShellAssistant(\n    client=client,\n    model="meta-llama/Meta-Llama-3.1-70B-Instruct")'

In [ ]:
result = assistant.chat('Get the names of the files in the current directory', verbose=True)
result

('The names of the files in the current directory are:\n\n- a-file.txt\n- foxes.txt\n- nebius_api_key\n- openai_api_key\n- sample_data',
 [{'role': 'user',
   'content': 'Get me the names of the files in the current directory'},
  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zd9lbBXKV7JDObcKkyTvbmwL', function=Function(arguments='{"command":"ls"}', name='execute_shell_command'), type='function')]),
  {'role': 'tool',
   'tool_call_id': 'call_zd9lbBXKV7JDObcKkyTvbmwL',
   'content': '{"success": true, "stdout": "a-file.txt\\nfoxes.txt\\nnebius_api_key\\nopenai_api_key\\nsample_data\\n", "stderr": "", "return_code": 0}'}])

In [ ]:
ChatCompletionMessage(
    content=None, refusal=None, role='assistant', audio=None, function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_zd9lbBXKV7JDObcKkyTvbmwL',
            function=Function(arguments='{"command":"ls"}', name='execute_shell_command'),
            type='function')
        ]
    )

In [ ]:
print(result[0])

The names of the files in the current directory are:

- a-file.txt
- foxes.txt
- nebius_api_key
- openai_api_key
- sample_data


Now, we will create two text files. One of them will contain a sonnet, and the other just some text.

In [ ]:
%%writefile foxes.txt
In twilight’s hush, the foxes roam the streets,
Through alleys dim and parks where shadows lie.
Their russet coats brush past the city’s feats,
Beneath the tower’s glare and amber sky.

With stealth, they weave through lanes and cobbled stone,
As silent hunters of the urban night.
Unseen by day, yet claiming streets their own,
In twilight’s cloak, they prowl till morning light.

They scavenge scraps left by the city’s din,
A wily dance of survival and grace.
Though wild at heart, they fit the world they're in,
An untamed spirit finding urban space.

And as the city sleeps, they make their rounds,
Foxes of London—ghosts in city bounds.

Writing foxes.txt


In [ ]:
%%writefile a-file.txt
First line
Second line
Third line

Writing a-file.txt


Let's ask our agent to find which text file contains a sonnet and copy that file.

In [ ]:
result = assistant.chat('Check every *.txt file in the current directory, then copy those files that contain a sonnet and also tell me what is the topic of this sonnet.', verbose=True)
result

('The output indicates that the command failed with a return code of 1. The standard error message is "find: missing argument to `-name\'". This means that there is a syntax error in the `find` command.\n\nTo fix this, you need to provide a valid pattern for the `-name` option. For example, to find all `.txt` files, you can use the following command:\n\n`find . -maxdepth 1 -type f -name "*.txt"`\n\nThis will find all files with the `.txt` extension in the current directory and its subdirectories. However, this alone will not tell you which files contain a sonnet or what the topic of the sonnet is.\n\nTo achieve that, you would need to use a more complex command or a script that can analyze the contents of the files. One possible approach is to use `grep` to search for specific keywords or patterns that are characteristic of sonnets, and then use a natural language processing tool to analyze the content and determine the topic.\n\nHere is an example of a more complex command that uses `

In [ ]:
result = assistant.chat('Add another sonnet to the contents of the sonnet_copy.txt.', verbose=True)
result

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mb4TqXf2TgYxhBHl9TjPmo7n', function=Function(arguments='{"file_path":"sonnet_copy.txt"}', name='read_file'), type='function')])


('I have successfully added another sonnet to the contents of the `sonnet_copy.txt` file. The new sonnet reflects on the beauty of the night and the enduring light within our hearts. If you need anything else, feel free to ask!',
 [{'role': 'user',
   'content': 'Add another sonnet to the contents of the sonnet_copy.txt.'},
  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mb4TqXf2TgYxhBHl9TjPmo7n', function=Function(arguments='{"file_path":"sonnet_copy.txt"}', name='read_file'), type='function')]),
  {'role': 'tool',
   'tool_call_id': 'call_mb4TqXf2TgYxhBHl9TjPmo7n',
   'content': '{"success": true, "content": "In realms of code, where logic meets the sky,\\nThere dwells a force both clever and refined,\\nA Nebius-born spark, an AI high,\\nThat weaves through data with a boundless mind.\\n\\nWith wisdom vast, it scours the cosmic spread,\\nFrom depths of learning, patterns clear and

In [ ]:
In twilight’s hush, the foxes roam the streets,
Through alleys dim and parks where shadows lie.
Their russet coats brush past the city’s feats,
Beneath the tower’s glare and amber sky.

With stealth, they weave through lanes and cobbled stone,
As silent hunters of the urban night.
Unseen by day, yet claiming streets their own,
In twilight’s cloak, they prowl till morning light.

They scavenge scraps left by the city’s din,
A wily dance of survival and grace.
Though wild at heart, they fit the world they're in,
An untamed spirit finding urban space.

And as the city sleeps, they make their rounds,
Foxes of London—ghosts in city bounds.

In [ ]:
In twilight’s hush, the foxes roam the streets,
Through alleys dim and parks where shadows lie.
Their russet coats brush past the city’s feats,
Beneath the tower’s glare and amber sky.

With stealth, they weave through lanes and cobbled stone,
As silent hunters of the urban night.
Unseen by day, yet claiming streets their own,
In twilight’s cloak, they prowl till morning light.

They scavenge scraps left by the city’s din,
A wily dance of survival and grace.
Though wild at heart, they fit the world they're in,
An untamed spirit finding urban space.

And as the city sleeps, they make their rounds,
Foxes of London—ghosts in city bounds.

In [ ]:
!md5sum *.txt

7b570f5b9d30db21cadd8730fd2a8806  a-file.txt
a6c90bc2d9c198facec986e01a79764e  foxes_sonnet.txt
a6c90bc2d9c198facec986e01a79764e  foxes.txt
